In [12]:
import ROOT as r
from pyHepMC3 import HepMC3 as hm
from array import array
from tqdm import tqdm
from pyHepMC3.rootIO import HepMC3 as hmrootIO

## File Format Conversion

This is the testing site which helps in file format converisons. Currently these are the conversions being supported :

- HepMC
- Root
- LHEF
- ASCII
- HDF5

In [19]:
r.gSystem.Load("libExRootAnalysis.so")

1

In [14]:
class Convert():
    def __init__(self):
        self.input_file = None
        self.output_file = None

    def add_input_file(self,input_file):
        self.input_file = input_file

    def add_output_file(self,output_file):
        self.output_file = output_file

    def convert_hepMC2_root(self):
        input_hep = hm.ReaderAsciiHepMC2(self.input_file)
        
        if input_hep.failed():
            raise ValueError('ERROR : Could not read HepMC2 file successfully.')

        output_root = hmrootIO.WriterRootTree(self.output_file)
        if output_root.failed():
            raise ValueError('ERROR : Could not open ROOT file successfully.')

        while not input_hep.failed():
            evt = hm.GenEvent()
            input_hep.read_event(evt)
            if input_hep.failed():
                print("INFO : File successfully converted")
                break

            output_root.write_event(evt)
            evt.clear()
        
        input_hep.close()
        output_root.close()

    def convert_root_hepMC2(self):
        input_root = hmrootIO.ReaderRootTree(self.input_file)
        if input_root.failed():
            raise ValueError('ERROR : Could not read ROOT file successfully.')

        output_hep = hm.WriterAsciiHepMC2(self.output_file)
        if output_hep.failed():
            raise ValueError('ERROR : Could not open HepMC2 file successfully.')
        
        while not input_root.failed():
            evt = hm.GenEvent()
            input_root.read_event(evt)
            if input_root.failed():
                print("INFO : File successfully converted")
                break
            output_hep.write_event(evt)
            evt.clear()
        
        input_root.close()
        output_hep.close()

    def convert_lhe_root(self):
        events = []
        event = ""
        in_event = False
        with open(self.input_file, "r") as fhin:
            for line in tqdm(fhin):
            
                if in_event and line.startswith("<"):
                    in_event = False
                    events.append(event)
                    event = ""
    
                if in_event:
                    event += line
    
                if line.startswith("<event>"):
                    in_event = True
    
            if event:
                events.append(event)
    
        f1 = r.TFile(self.output_file, "recreate")
        t1 = r.TTree("t","t")
    
        pdgid = array( 'i', [ 0 ] )
        bevent = array( 'i', [ 0 ] )
        status = array( 'i', [ 0 ] )
        parent1 = array( 'i', [ 0 ] )
        parent2 = array( 'i', [ 0 ] )
        color1 = array( 'i', [ 0 ] )
        color2 = array( 'i', [ 0 ] )
        mass = array( 'd', [ 0 ] )
        spin = array( 'd', [ 0 ] )
        p4 = r.TLorentzVector(1,1,0,5)
    
        t1.Branch("id",pdgid, "id/I")
        t1.Branch("event",bevent, "event/I")
        t1.Branch("status",status, "status/I")
        t1.Branch("parent1",parent1, "parent1/I")
        t1.Branch("parent2",parent2, "parent2/I")
        t1.Branch("color1",color1, "color1/I")
        t1.Branch("color2",color2, "color2/I")
        t1.Branch("mass",mass, "mass/F")
        t1.Branch("spin",spin, "spin/F")
        t1.Branch("p4.","TLorentzVector",p4)
    
        for ievt,evt in tqdm(enumerate(events)):
            particle_lines = evt.splitlines()[1:]
            for particle_line in particle_lines:
                parts = particle_line.split()
                evt_pdgid = int(parts[0])
                evt_status = int(parts[1])
                evt_parent1 = int(parts[2])
                evt_parent2 = int(parts[3])
                evt_color1 = int(parts[4])
                evt_color2 = int(parts[5])
                evt_px, evt_py, evt_pz, evt_e = map(float,parts[6:10])
                evt_mass = float(parts[10])
                evt_spin = float(parts[11])
    
                p4.SetPxPyPzE(evt_px, evt_py, evt_pz, evt_e)
    
                bevent[0] = ievt
                pdgid[0] = evt_pdgid
                status[0] = evt_status
                parent1[0] = evt_parent1
                parent2[0] = evt_parent2
                color1[0] = evt_color1
                color2[0] = evt_color2
                mass[0] = evt_mass
                spin[0] = evt_spin
    
                t1.Fill()
    
        t1.Print()
        t1.Write()
        f1.Close()

    def sanitize_format(self,file_format):
        return file_format.lower()

    def get_convert_table(self):
        self.convert_dict = {
            'root' : {
                'hepmc2' : self.convert_root_hepMC2
                },
            'hepmc2' : {
                'root' : self.convert_hepMC2_root
                },
            'lhe' : {
                'root' : self.convert_lhe_root
            }
            }

    def convert_menu(self,input_file,output_file,input_format,output_format):
        self.add_input_file(input_file)
        self.add_output_file(output_file)
        self.get_convert_table()

        input_format = self.sanitize_format(input_format)
        output_format = self.sanitize_format(output_format)

        self.convert_dict[input_format][output_format]()



In [15]:
c1 = Convert()
c1.convert_menu('root_output.root','lhe_output.lhe','ROOT','LHE')

0it [00:00, ?it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 6: invalid continuation byte

In [ ]:
r